In [7]:
import numpy as np

In [31]:
aircraft = {'Vtd_m/s': 25.191, 'tr_s': 3, 'g_m/s2': 9.807, 'rho_kg/m3': 1.225,
            'wingArea_m2': 16.72, 'wingSpan_m': 11.02, 'mass_kg': 975, 'mu_R': 0.2, 
            'alpha_deg': 0, 'LDmax': 11}

environment = {'accelDueToGrav_m/s2': 9.807, 'densityOfAir_kg/m3': 1.225, 'rollingFrictionCoef': 0.2}

In [32]:
def get_CL(m, g, rho, V, S):
    return (m*g)/(0.5*rho*V**2*S)

def get_A(b, S):
    A = b**2/S
    return A

def get_oswaldEff(b, S):
    A = get_A(b, S)
    e = 1.78*(1-0.045*A**0.68)-0.64 # An estimation of the Oswald's Efficiency Factor
    return e

def get_K(b, S):
    A = get_A(b, S)
    e = get_oswaldEff(b, S)
    K = 1/(np.pi*A*e)
    return K

def get_CD0(LDmax, b, S):
    K = get_K(b, S)
    CD0 = 1/((2*LDmax)**2*K)
    return CD0

In [10]:
def get_ground_roll(aircraft, environment):
    Vtd = aircraft['Vtd_m/s']
    tr = aircraft['tr_s']
    LDmax = aircraft['LDmax']
    b = aircraft['wingSpan_m']
    S = aircraft['wingArea_m2']
    m = aircraft['mass_kg']

    g = environment['accelDueToGrav_m/s2']
    rho = environment['densityOfAir_kg/m3']
    mu = environment['rollingFrictionCoef']

    W = m*g

    CL = get_CL(m, g, rho, 
                Vtd, S)
    CD0 = get_CD0(LDmax, b, S)
    K = get_K(b, S)
    alpha = aircraft['alpha_deg']

    ground_roll = Vtd*tr+(1/(2*g*((rho*S)/(2*W))*(mu*CL-CD0-K*CL**2)))*np.log((-np.sin(alpha)-mu*np.cos(alpha))/(-np.sin(alpha)-mu*np.cos(alpha)+((rho*S)/(2*W))*(mu*CL-CD0-K*CL**2)*Vtd**2))
    return ground_roll

def get_field_dimensions(aircraft, environment, lSF, wSF):
    length = get_ground_roll(aircraft, environment)*lSF
    width =  aircraft['wingSpan_m']*wSF
    area = length * width
    return length, width, area

In [33]:
get_ground_roll(aircraft, environment)

295.78036849940406

In [34]:
length_safety_factor = 1.15
width_safety_factor = 3

length, width, area = get_field_dimensions(aircraft, environment, length_safety_factor, width_safety_factor)

print("Length of field (m):", f'{length:.1f}')
print("Width of field (m):", f'{width:.1f}')
print("Area of field (m2):", f'{area:.1f}')

Length of field (m): 340.1
Width of field (m): 33.1
Area of field (m2): 11245.3
